Wczytanie danych

In [131]:
import pandas as pd
import numpy as np

In [132]:
with open('mammographic_masses.names') as f:
    print(f.read())

1. Title: Mammographic Mass Data

2. Sources:

   (a) Original owners of database:
        Prof. Dr. Rüdiger Schulz-Wendtland
        Institute of Radiology, Gynaecological Radiology, University Erlangen-Nuremberg
        Universitätsstraße 21-23
        91054 Erlangen, Germany
        
   (b) Donor of database:
        Matthias Elter
        Fraunhofer Institute for Integrated Circuits (IIS)
        Image Processing and Medical Engineering Department (BMT) 
        Am Wolfsmantel 33
        91058 Erlangen, Germany
        matthias.elter@iis.fraunhofer.de
        (49) 9131-7767327 
        
   (c) Date received: October 2007
 
3. Past Usage:
    M. Elter, R. Schulz-Wendtland and T. Wittenberg (2007)
    The prediction of breast cancer biopsy outcomes using two CAD approaches that both emphasize an intelligible decision process.
    Medical Physics 34(11), pp. 4164-4172

4. Relevant Information:
    Mammography is the most effective method for breast cancer screening
    available today

In [133]:
data = pd.read_csv('mammographic_masses.data', header=None)
data.head()

,0,1,2,3,4,5
0,5,67,3,5,3,1
1,4,43,1,1,?,1
2,5,58,4,5,3,1
3,4,28,1,1,3,0
4,5,74,1,5,?,1


In [134]:
data = data.rename(columns={0: 'BI-RADS assessment', 1 : 'Age', 2 : 'Shape', 3 : 'Margin', 4 : 'Density', 5 : 'Severity'})

In [135]:
data.head()

,BI-RADS assessment,Age,Shape,Margin,Density,Severity
0,5,67,3,5,3,1
1,4,43,1,1,?,1
2,5,58,4,5,3,1
3,4,28,1,1,3,0
4,5,74,1,5,?,1


Sprawdzenie czy są brakujące dane

In [136]:
print(data.isnull().values.any())

False


Widać że niektóre wartości są pouzupełniane stringami, mimo że wyszukanie pustych komórek zwraca false.
Puste wartości są stringami w postaci '?'

In [137]:
print("Liczba wartosci {} wynosi {}".format('?',np.count_nonzero(data.values == '?')))

Liczba wartosci ? wynosi 162


Usunięcie wszystkich wartości gdzie jest "?"

In [138]:
for column in data.columns:
    to_mean_calc = np.array(data.loc[data[column] != '?', column])
    to_mean_calc = to_mean_calc.astype('int32')
    data.loc[data[column] == '?', column] = np.mean(to_mean_calc)

print("Liczba wartosci {} wynosi {}".format('?',np.count_nonzero(data.values == '?')))

Liczba wartosci ? wynosi 0
